In [25]:
import numpy as np
import pandas as pd
from datetime import date
import collections
import datetime
import os
import xarray as xr
import csv

modelnames_OG = ['BCC-CSM2-MR', 'CAMS-CSM1-0', 'CESM2', 'CESM2-WACCM', 'EC-Earth3', 'EC-Earth3-Veg', 'FGOALS-f3-L', 
                 'GFDL-ESM4', 'INM-CM4-8','INM-CM5-0', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorESM2-MM']

modelnames_py = ['BCC-CSM2-MR','CESM2','CESM2-WACCM','EC-Earth3','EC-Earth3-Veg','FGOALS-f3-L','GFDL-ESM4',
              'INM-CM4-8','INM-CM5-0','MPI-ESM1-2-HR','MRI-ESM2-0', 'NorESM2-MM']

modelnames_glo = ['BCC-CSM2-MR','CAMS-CSM1-0','CESM2','CESM2-WACCM','EC-Earth3','EC-Earth3-Veg','FGOALS-f3-L',
                  'GFDL-ESM4','INM-CM4-8','INM-CM5-0','MPI-ESM1-2-HR','MRI-ESM2-0', 'NorESM2-MM']

SSPs = ['ssp126','ssp245','ssp370','ssp585']

In [26]:
#All of our basins
basins_all = {'RHINE':'6242', 'RHONE':'6243','PO':'6241', 'DANUBE':'6202', 'TITICACA':'3912', 'SANTA':'3425', 
            'OCONA':'3418', 'MAJES':'3416', 'MAGDALENA':'3227', 'AMAZON':'3203', 'YELCHO':'3429', 
            'VALDIVIA':'3428', 'SERRANO':'3426', 'RAPEL':'3423', 'PUELO':'3422', 'PASCUA':'3420', 
            'PALENA':'3419', 'HUASCO':'3412', 'COPIAPO':'3409', 'CISNES':'3408', 'BIOBIO':'3405', 'BAKER':'3404',
            'AZOPARDO':'3403', 'AISEN':'3401', 'SANTA CRUZ':'3244', 'NEGRO':'3232', 'COLORADO':'3212', 
            'CHICO':'3209', 'TORNEALVEN':'6255', 'THJORSA':'6254', 'OLFUSA':'6237', 'LULEALVEN':'6227', 
            'KUBAN':'6223', 'KALIXALVEN':'6219', 'GLOMAA':'6213', 'DRAMSELVA':'6209', 'SVARTA':'6110', 
            'LAGARFLJOT':'6104', 'JOKULSA A FJOLLUM':'6101', 'CLUTHA':'5406', 'YUKON':'4435', 'TAKU':'4431', 
             'SUSITNA':'4430','STIKINE':'4428', 'SKEENA':'4427','SKAGIT':'4426','NUSHAGAK':'4418','NASS':'4416',
            'KUSKOKWIM':'4414','FRASER':'4410', 'COPPER':'4408', 'COLUMBIA':'4406', 'ALSEK':'4401', 'NELSON':'4125', 
              'MACKENZIE':'4123','COLVILLE':'4110', 'YSYK-KOL':'2919', 'UVS NUUR':'2918', 'TARIM HE':'2914', 
              'TALAS':'2913', 'LAKE BALKHASH':'2910','HAR US NUUR':'2909', 'CHUY':'2905', 'ARAL SEA':'2902', 
              'YELLOW RIVER':'2434', 'MEKONG':'2421', 'KAMCHATKA':'2413', 'SALWEEN':'2319', 'IRRAWADDY':'2310', 
              'INDUS':'2309', 'GANGES':'2306','BRAHMAPUTRA':'2302', 'OB':'2108', 'INDIGIRKA':'2103','YANGTZE' : '2433'}

#### GloGEM

In [41]:
#Loading in area data
fpathglo = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/Lizz Research Stuff/Runoff-intercomparison/GloGEM-output/'

RGIregionsGlo = ['RGI01-Alaska', 'RGI02-WesternCanada', 'RGI03-ArcticCanadaN', 'RGI04-ArcticCanadaS', 
                 'RGI05-Greenland', 'RGI06-Iceland', 'RGI07-Svalbard', 'RGI08-Scandinavia',
                 'RGI09-RussianArctic', 'RGI10-NorthAsia', 'RGI11-CentralEurope','RGI12-Caucasus', 
                 'RGI13-CentralAsiaN', 'RGI14-SouthAsiaWest', 'RGI15-SouthAsiaEast', 'RGI16-LowLatitudes', 
                 'RGI17-SouthernAndes', 'RGI18-NewZealand']

regional_fnames = ['alaska', 'westerncanada', 'arcticcanadaN', 'arcticcanadaS', 'greenland', 'iceland',
                   'svalbard', 'scandinavia', 'russianarctic', 'northasia', 'centraleurope','caucasus', 'centralasiaN', 'centralasiaW', 
                   'centralasiaS', 'lowlatitudes', 'southernandes', 'newzealand']

In [28]:
#Loading all area data
all_area_dfs = []

# Code to import single GCM/SSP areas (which are equivalent in yr 2000)
for r, region in enumerate(RGIregionsGlo):
    regionnumber = str(region[3:5])
    region_area = []
    temp_df = pd.read_csv(fpathglo + region + '/files/BCC-CSM2-MR/ssp585/' + regional_fnames[r] + '_Area_r1.dat', sep='\s+', index_col="ID")
    temp_df.index = temp_df.index.map(lambda x: str(x).zfill(5))
    temp_df.index = regionnumber + '.' + temp_df.index.astype(str)
    region_area.append(temp_df)
    all_area_dfs.extend(region_area)

# Concatenate all the DataFrames in the list into one
combined_area_df = pd.concat(all_area_dfs)

In [30]:
import json
def select_glaciers_json(basin='all'):
    '''
    Select glaciers within a basin by MRBID from a json-file,
    which is stored in the data directory.

    Args:
    -----
    basin: str
        String of MRBID or 'all'.

    Returns:
    --------
    If basin is 'all' a list of all relevant glaciers is returned, for
    initiating glacier simulations. If basin is a MRBID the list of glaciers
    within that basin is returned.
    
    Copy of a function written by Erik Holmgren (2022) in holmgren_gha.utils
    '''

    # fpath = './data/rgi_ids_per_basin.json'
    fpath = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/Lizz Research Stuff/rgi_ids_per_basin.json'  
    with open(fpath) as f:
        basin_dict = json.load(f)

    if basin.lower() != 'all':
        glacier_list = basin_dict[basin]
    else:
        glacier_list = list(itertools.chain.from_iterable(basin_dict.values()))

    return glacier_list

In [31]:
def sum_basin_area(basin_RGI_list, area_data):
    # Create new list to match our RGI formatting
    new_basin_list = [str(x)[-8:] for x in basin_RGI_list]
    
    # Filter new_basin_list to keep only the indexes present in the DataFrame
    new_basin_list = [x for x in new_basin_list if x in area_data.index]
    
    # Extract glaciers contained in the list from original df and create a new df
    new_df = area_data.loc[new_basin_list].copy()
    # print(new_df)
    
    # Sum the values of the glaciers within the basin
    summed_basin_area = new_df.sum()
    #print(summed_basin_runoff)
    
    return summed_basin_area

In [32]:
#Generating the list of glaciers within each basin 
basin_gls = {}
glaciers_in_basin_by_region = {}
baseline_area = {}
for basin, ID in basins_all.items():
    basin_gls[basin] = select_glaciers_json(ID)

In [42]:
#Summing basins
basin_area_sums_glo = {}
for basin, ID in basins_all.items():
    basin_area_sums_glo[basin] = sum_basin_area(select_glaciers_json(basins_all[basin]), combined_area_df)

In [36]:
#Creating our df
initial_basin_areas_glo = {}
for b, basin in enumerate(basins_all):
    initial_basin_areas_glo[basin] = basin_area_sums_glo[basin][20]

area_df_glo = pd.DataFrame({'GloGEM': initial_basin_areas_glo})
area_df_glo.index.name = 'Basin'

In [40]:
area_df_glo.loc['INDUS']

GloGEM    27211.115
Name: INDUS, dtype: float64

In [117]:
#Exporting as a CSV to use for larger basins where reading in both discharge and area crashes my laptop
output_dir = '/Users/finnwimberly/Desktop/Lizz Research/CSV Outputs/Area/All Shared Basins/'

fname = 'InitialAreas_GloGEM.csv'

# Define the full path of the output file
output_path = os.path.join(output_dir, fname)

# Save the DataFrame as CSV
area_df_glo.to_csv(output_path, header=True, index=True)

#### PyGEM

In [120]:
#Importing all runoff data, taking annual sum, and converting m^3 to km^3
import glob   #use glob to group files by filename similarities (in this case, SSP)

RGIregionsPy = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', 
                '11', '12', '13', '14', '15', '16', '17', '18']

fpathPy1 = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/'
fpathPy2 = 'Lizz Research Stuff/Runoff-intercomparison/PyGEM/'

area_ds = {}

for r, region in enumerate(RGIregionsPy):
    fpath1 = '/{}/area_annual-ssp585/R{}_area_annual_c2_ba1_1set_2000_2100--'.format(region, region)
    file_pattern = f'{fpathPy1 + fpathPy2 + fpath1}*.nc'
    file_list = glob.glob(file_pattern)
    
    datasets = []  # Create an empty list for each SSP
    if file_list:
        for file in file_list: 
            with xr.open_dataset(file) as ds:
                ds = ds.glac_area_annual.load()
                datasets.append(ds)
    
        combined_ds = xr.concat(datasets, dim='glacier')  # Concatenate the datasets
        area_ds[region] = combined_ds * 1e-6  #m^2 to km^2


In [121]:
basin_gls = {}
for basin, ID in basins_all.items():
    basin_gls[basin] = select_glaciers_json(ID)

In [122]:
# Sorting into basins
basin_areas = {}
for basin, glacier_list in basin_gls.items():
    ## loop over them all, drop the irrelevant IDs, and concatenate the result
    basin_areas[basin] = {}
    for r, region in enumerate(RGIregionsPy):
        ds_list = []
        try:
            ds_filtered = area_ds[region].where(area_ds[region].RGIId.isin(glacier_list), drop=True)
            #print(ds_filtered)
            ds_list.append(ds_filtered)
        except ValueError: ## happens if there are no glaciers from this batch in the selected region
            continue
        basin_areas[basin][region] = xr.concat(ds_list, dim='glacier')  #Summing glaciers within basin

In [144]:
#Summing glaciers within basins 
regional_area_sums_py = {}
for b, basin in enumerate(basins_all):
    regional_area_sums_py[basin] = {}
    for r, region in enumerate(RGIregionsPy):
        regional_area_sums_py[basin][region] = basin_areas[basin][region].sum(dim='glacier')

#Combining basin sums for same basins that cross RGI region boundaries
basin_area_sums_py = {}
for b, basin in enumerate(basins_all):
    basin_area_sums_py[basin] = 0
    for r, region in enumerate(RGIregionsPy):
        basin_area_sums_py[basin] += regional_area_sums_py[basin].get(region, 0)

In [145]:
#Creating our df
#All models begin with same area so we just select the first one (BCC-CSM2-MR)
initial_basin_areas_py = {}
for b, basin in enumerate(basins_all):
    initial_basin_areas_py[basin] = basin_area_sums_py[basin].sel(year = 2000, model = 1).values

area_df_py = pd.DataFrame({'PyGEM': initial_basin_areas_py})
area_df_py.index.name = 'Basin'

In [146]:
area_df_py

,PyGEM
Basin,
AISEN,153.9163243842435
ALSEK,5499.486983319258
AMAZON,1400.1959543089079
ARAL SEA,12557.754489200304
AZOPARDO,30.207007901739672
...,...
YANGTZE,1710.2257143224706
YELCHO,238.95524261992426
YELLOW RIVER,177.38074014493043


#### OGGM

In [147]:
#Larger datasets so it helpful to break down into chunks
basins_europe = {'RHINE':'6242', 'RHONE':'6243','PO':'6241', 'DANUBE':'6202','TORNEALVEN':'6255', 
                 'THJORSA':'6254', 'OLFUSA':'6237', 'LULEALVEN':'6227', 'KUBAN':'6223', 
                 'KALIXALVEN':'6219', 'GLOMAA':'6213', 'DRAMSELVA':'6209', 'SVARTA':'6110', 
                'LAGARFLJOT':'6104', 'JOKULSA A FJOLLUM':'6101'}

basins_Namerica = {'YUKON':'4435', 'TAKU':'4431','SUSITNA':'4430','STIKINE':'4428', 
                   'SKEENA':'4427','SKAGIT':'4426','NUSHAGAK':'4418','NASS':'4416',
                   'KUSKOKWIM':'4414','FRASER':'4410', 'COPPER':'4408','COLUMBIA':'4406',
                    'ALSEK':'4401', 'NELSON':'4125', 'MACKENZIE':'4123','COLVILLE':'4110'}

basins_Samerica = {'TITICACA':'3912', 'SANTA':'3425', 
            'OCONA':'3418', 'MAJES':'3416', 'MAGDALENA':'3227', 'AMAZON':'3203', 'YELCHO':'3429', 
            'VALDIVIA':'3428', 'SERRANO':'3426', 'RAPEL':'3423', 'PUELO':'3422', 'PASCUA':'3420', 
            'PALENA':'3419', 'HUASCO':'3412', 'COPIAPO':'3409', 'CISNES':'3408', 'BIOBIO':'3405', 
            'BAKER':'3404','AZOPARDO':'3403', 'AISEN':'3401', 'SANTA CRUZ':'3244', 'NEGRO':'3232',
            'COLORADO':'3212', 'CHICO':'3209'}

basins_NZ = {'CLUTHA':'5406'}

basins_asia = {'YSYK-KOL':'2919', 'UVS NUUR':'2918', 'TARIM HE':'2914', 'TALAS':'2913', 
            'LAKE BALKHASH':'2910','HAR US NUUR':'2909', 'CHUY':'2905', 'ARAL SEA':'2902', 'YELLOW RIVER':'2434', 
          'MEKONG':'2421', 'KAMCHATKA':'2413', 'SALWEEN':'2319', 'IRRAWADDY':'2310', 'INDUS':'2309', 'GANGES':'2306',
          'BRAHMAPUTRA':'2302', 'OB':'2108', 'INDIGIRKA':'2103', 'YANGTZE' : '2433'}

In [148]:
#Generic filepath to navigate to Drive folder 
fpathOG1 = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/'
fpathOG2 = 'Lizz Research Stuff/Runoff-intercomparison/OGGM/lschuster/runs_2023.3/output/basins/'

#Importing area data, OGGM is grouped by basin
#here we are just doing Namerica
#had trouble loading data all at once so breaking it up into separate cells by region
area_ds = {}
for basin, ID in basins_Namerica.items():
    fpath_basin = 'gcm_from_2000_bc_2000_2019/{}/'.format(ID)
    #print(f'{fpathOG + fpath_basin} (1)/*.nc')
    with xr.open_mfdataset(f'{fpathOG1 + fpathOG2 + fpath_basin}/*.nc') as ds:
        ds = ds.area.load()
    area_ds[basin] = ds

In [149]:
#Importing S America area data
for basin, ID in basins_Samerica.items():
    fpath_basin = 'gcm_from_2000_bc_2000_2019/{}/'.format(ID)
    #print(f'{fpathOG + fpath_basin} (1)/*.nc')
    with xr.open_mfdataset(f'{fpathOG1 + fpathOG2 + fpath_basin}/*.nc') as ds:
        ds = ds.area.load()
    area_ds[basin] = ds

In [150]:
#Importing Europe area data
for basin, ID in basins_europe.items():
    fpath_basin = 'gcm_from_2000_bc_2000_2019/{}/'.format(ID)
    #print(f'{fpathOG + fpath_basin} (1)/*.nc')
    with xr.open_mfdataset(f'{fpathOG1 + fpathOG2 + fpath_basin}/*.nc') as ds:
        ds = ds.area.load()
    area_ds[basin] = ds

In [151]:
#Importing NZ area data
for basin, ID in basins_NZ.items():
    fpath_basin = 'gcm_from_2000_bc_2000_2019/{}/'.format(ID)
    #print(f'{fpathOG + fpath_basin} (1)/*.nc')
    with xr.open_mfdataset(f'{fpathOG1 + fpathOG2 + fpath_basin}/*.nc') as ds:
        ds = ds.area.load()
    area_ds[basin] = ds

In [152]:
#Importing Asia area data
for basin, ID in basins_asia.items():
    fpath_basin = 'gcm_from_2000_bc_2000_2019/{}/'.format(ID)
    #print(f'{fpathOG + fpath_basin} (1)/*.nc')
    with xr.open_mfdataset(f'{fpathOG1 + fpathOG2 + fpath_basin}/*.nc') as ds:
        ds = ds.area.load()
    area_ds[basin] = ds

In [153]:
#Summing individual glacier runoff into basin totals and converting m^2 to km^2
basin_area_sums_OG = {}
for basin, ID in basins_all.items():
    basin_area_sums_OG[basin] = area_ds[basin].sum(dim = 'rgi_id') * 1e-6

In [154]:
#Creating our df
#All models begin with same area so we just select the first GCM (BCC-CSM2-MR) AND ssp126
initial_basin_areas_OG = {}
for b, basin in enumerate(basins_all):
    initial_basin_areas_OG[basin] = basin_area_sums_OG[basin].sel(time = 2000, gcm = 'BCC-CSM2-MR', scenario = 'ssp126').values

area_df_OG = pd.DataFrame({'OGGM': initial_basin_areas_OG})
area_df_OG.index.name = 'Basin'

In [155]:
area_df_OG

,OGGM
Basin,
AISEN,157.47366
ALSEK,5614.803
AMAZON,1490.8467
ARAL SEA,13309.696
AZOPARDO,31.184336
...,...
YANGTZE,1741.5363
YELCHO,247.73232
YELLOW RIVER,183.76598


### Creating Single DataFrame 
GloGEM lacks regions 8 and 6 (Scandinavia/Iceland) so we trim these ten basins from the PyGEM and OGGM dataframes

In [160]:
#Combining into a single dataframe 
initial_areas_df = pd.concat([area_df_glo, area_df_py, area_df_OG], axis=1)
#initial_areas_df = pd.concat([area_df_glo, area_df_py, area_df_OG], axis=1, join='inner')
initial_areas_df

,GloGEM,PyGEM,OGGM
Basin,,,
AISEN,154.120,153.9163243842435,157.47366
ALSEK,5498.554,5499.486983319258,5614.803
AMAZON,1402.745,1400.1959543089079,1490.8467
ARAL SEA,12522.451,12557.754489200304,13309.696
AZOPARDO,30.165,30.207007901739672,31.184336
...,...,...,...
YANGTZE,1647.631,1710.2257143224706,1741.5363
YELCHO,238.940,238.95524261992426,247.73232
YELLOW RIVER,177.422,177.38074014493043,183.76598


In [161]:
#Exporting as a CSV
output_dir = '/Users/finnwimberly/Desktop/Lizz Research/CSV Outputs/Area/All Shared Basins/'

fname = 'InitialAreas_ALL.csv'

# Define the full path of the output file
output_path = os.path.join(output_dir, fname)

# Save the DataFrame as CSV
initial_areas_df.to_csv(output_path, header=True, index=True)